# MUHC Vehicule Routing Optimization
### Group 2

**This project aims to optimize the delivery routes for the McGill University Health Center (MUHC) across multiple healthcare sites.**  

The primary goals are to:

- **Minimize transportation costs**
- **Reduce the distance traveled**, which in turn lowers fuel consumption and CO2 emissions
- **Ensure all demands** from each healthcare site are met efficiently

The routes are optimized using **Gurobi**, a powerful mathematical optimization solver, based on key operational constraints such as **truck capacity** (both volume and weight), **site demand**, and **priority** assigned to different locations.


In addition to solving the baseline model, a **sensitivity analysis** is conducted to explore how adjustments in key variables impact the overall results. These variables include:

- **Truck Size**: Evaluating how changes in truck capacity affect costs and delivery efficiency.
- **Time vs. Distance Weighting**: Adjusting the weight assigned to minimizing time versus distance to assess trade-offs.
- **Site Prioritization**: Investigating how prioritizing certain healthcare sites impacts the overall routing strategy.


- **Minimization of Distance and Time**  
  The model seeks to reduce the total kilometers traveled and time spent on deliveries, improving overall efficiency.

- **Efficient Resource Utilization**  
  Ensuring trucks are fully utilized in terms of both weight and volume, maximizing the capacity of each journey.

- **CO2 Emission Reduction**  
  Promoting environmentally friendly logistics by optimizing routes to reduce fuel consumption and emissions.


**This approach ensures that the MUHC logistics network operates more efficiently**, reducing both **operational costs** and **environmental impact** while maintaining high service levels.


### Imported Libraries

In [1]:
# Import libraries

import gurobipy as gb
from gurobipy import GRB
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import folium as fl
import random
import requests
from folium.features import DivIcon
from gurobipy import *
from IPython.display import display


### Our Business Problem and Data
The McGill University Health Center (MUHC) is one of the largest healthcare networks in Canada, comprising multiple sites: Glen (GL), Allen Memorial (AM), Lachine (LCH), Montreal General Hospital (MGH), Montreal Neurological Hospital (MNH), and Royal Victoria (RC). The challenge faced by MUHC is to efficiently distribute medical supplies across these sites while minimizing transportation costs, fuel consumption, and CO2 emissions.

The problem we are addressing can be classified as a Vehicle Routing Problem (VRP), which seeks to determine the optimal routes for a fleet of trucks delivering goods to a set of locations. The trucks must adhere to various constraints, such as weight and volume capacity, while ensuring that the demands of each healthcare site are met. Our goal is to develop a model that finds the best possible routes, taking into account both distance and time, as well as the priority of each site.

We utilize the Folium library to create an interactive map that visualizes all sites for the MUHC logistics medical supply network. Each healthcare site is marked by its specific coordinates.

In [2]:
# The six healthcare sites are indexed as follows:
# indices
indexes = {
    0: 'GL',
    1: 'AM',
    2: 'LCH',
    3: 'MGH',
    4: 'MNH',
    5: 'RC'
}

# The following truck will serve as a baseline for our model. Further trucks will be considered for analysis
# truck data
truck_data = {
    "truck": ["10'", "15'", "17'", "20'", "26'"],
    "max_volume": [402, 764, 865, 1016, 1682],
    "max_load": [2850, 6385, 6160, 5700, 12859]
}
# Baseline max volume and load parameters for 10 ft cube van
max_volume = truck_data["max_volume"][0]
max_load = truck_data["max_load"][0]

# The following demand and priority are required metrics for our problem, they will be static throughout optimizations.
# demand
volume_cuft = [0, 310, 70, 25, 15, 102] 
load_lbs = [0, 2200, 550, 60, 40, 720]

# priority
priority = [0, 2, 1, 1, 1, 2]


# The travel times and distances between the healthcare centers are imported from an Excel file:
#excel_path = '/Users/meg-annbrosseau/Desktop/McGill/MMA/Fall 2024/MGSC 662/Group Project/travel_time_muhc_project.xlsx'
excel_path = 'travel_time_muhc_project.xlsx'
sites = pd.read_excel(excel_path , sheet_name = "Sites")
travel_time_no_traffic = pd.read_excel(excel_path , sheet_name = "travel_time_notraffic", index_col=0)
distance = pd.read_excel(excel_path , sheet_name = "travel_distances", index_col=0)

print("Average Travel Time Without Traffic:")
display(travel_time_no_traffic)
print("Average Distance Between Centers:")
display(distance)

# Coordinates for each site
coordinates = {
    0: [45.47317, -73.60596],
    1: [45.52118, -73.55575],
    2: [45.4375, -73.67794],
    3: [45.49646, -73.590492],
    4: [45.50773, -73.57914],
    5: [45.510735, -73.578243]    
    }

loc_name = list(indexes.values())

# Set the width and height for the map to control the size
m = fl.Map(location=coordinates[0], zoom_start=12)

# Add point markers for each location
for name, loc in coordinates.items():
    fl.Marker(location=loc, popup=loc_name[name]).add_to(m)

# Add bold text for each location near the point marker
for loc in coordinates:
    text = loc_name[loc]
    lat = coordinates[loc][0]
    lon = coordinates[loc][1]
    
    fl.map.Marker(
        [lat + 0.001, lon + 0.001],  # Slight offset to prevent overlap with point marker
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12pt; font-weight: bold;"><b>%s</b></div>' % text,  # Bold text for locations
        )
    ).add_to(m)

# No lines between the locations; only markers and bold text

m

Average Travel Time Without Traffic:


,GL,AM,LCH,MGH,MNH,RC
GL,0.0,10.8,18.0,8.4,9.6,12.0
AM,10.8,0.0,20.4,4.8,2.4,4.8
LCH,18.0,20.4,0.0,19.2,20.4,21.6
MGH,8.4,4.8,19.2,0.0,2.4,6.0
MNH,9.6,2.4,20.4,2.4,0.0,7.2
RC,12.0,4.8,21.6,6.0,7.2,0.0


Average Distance Between Centers:


,GL,AM,LCH,MGH,MNH,RC
GL,0.0,8.5,13,7,7.0,10.0
AM,8.0,0.0,15,3,2.0,4.0
LCH,14.0,15.0,0,14,17.0,19.0
MGH,6.0,3.5,16,0,2.5,4.5
MNH,7.5,2.5,18,2,0.0,5.5
RC,9.0,4.5,16,4,5.0,0.0


### Parameters, Indices, and Auxiliary Variables

#### 1. Truck Capacity:
- $ C_{\text{volume}} $: Maximum volume capacity of the truck, i.e., the total volume the truck can carry during a journey.
  $$
  C_{\text{volume}} > 0
  $$

- $ C_{\text{weight}} $: Maximum weight capacity of the truck, i.e., the total weight the truck can carry during a journey.
  $$
  C_{\text{weight}} > 0
  $$

#### 2. Demand:
- $ v_j $: The demand (in volume cuft) at center $ j $.
  $$
  v_j \geq 0, \quad \forall j
  $$

- $ w_j $: The demand (in weight lbs) at center $ j $.
  $$
  w_j \geq 0, \quad \forall j
  $$

#### 3. Distance and Time:
- $ d_{ij} $: The distance between center $ i $ and center $ j $.
  $$
  d_{ij} \geq 0, \quad \forall i, j
  $$

- $ d_{\text{max}} $: The maximum possible distance between any two centers, used for normalizing the distance.
  $$
  d_{\text{max}} > 0
  $$


- $ t_{\text{max}} $: The maximum travel time between any two centers, used for normalizing the travel time.
  $$
  t_{\text{max}} > 0
  $$

#### 4. Objective Function Weights:
- $ \lambda_1 $ and $ \lambda_2 $: Weights balancing the importance of minimizing distance vs. time in the objective function.
  $$
  \lambda_1 + \lambda_2 = 1, \quad \lambda_1 \in [0, 1], \quad \lambda_2 \in [0, 1]
  $$
Where $ \lambda_1 + \lambda_2 = 1$

- $ \gamma $: A parameter that controls the trade-off between minimizing the cost (distance and time) and respecting the priority of servicing centers.
  $$
  \gamma \in [0, 1]
  $$

#### 5. Priority:

- $ p_j $: The priority of servicing center $ j $. Higher $ p_j $ values represent higher priority.
  $$
  p_j \geq 0, \quad \forall j
  $$

#### 6. Other

- $ N $: The total number of centers (including the depot).
- $ Z $: The total number of traffic time periods.
- $ i, j $: Indices for the centers.
- $ K $: The maximum number of journeys the truck can make in total.
  $$
   K = max ( \lceil \frac{\sum_{j=1}^{n} v_j}{C_{\text{volume}}} \rceil, \lceil \frac{\sum_{j=1}^{n} w_j}{C_{\text{weight}}} \rceil ) + \text{buffer} 
  $$
- $ k $: Index for the journeys.
- $ MinOrder_{k} $: Min order for a node in journey k
- $ MaxOrder_{k} $: Max order for a node in journey k



In [3]:

# Definitions of constants
# Number of locations 
N = distance.shape[0]

# Define starting location
START_LOCATION = 'GL'
START_NODE_IDX = distance.index.get_loc(START_LOCATION)

# Maximum distance, for standardization
max_dist = distance.max().max()

# Maximum time, for standardization
max_time = travel_time_no_traffic.max().max()

# Maximum number of journeys a driver can make
def obtain_k(max_volume_k, max_load_k):
    buffer = 0
    number_vol = math.ceil(sum(volume_cuft)/max_volume_k) + buffer 
    number_w = math.ceil(sum(load_lbs)/max_load_k) + buffer 
    K = max(number_w,number_vol)
    return K

K = obtain_k(max_volume,max_load)

# The following weights will be subject to further analysis. 
# Baseline Weight attributed to distance vs time, cost variables of our optimization 
l = 0.75
# Baseline Weight attributed to costs vs respect of priorties
g = 0.25



### The model
The model for optimizing MUHC's vehicle routing is constructed using a mathematical programming approach, formulated as a Mixed-Integer Linear Programming (MILP) problem. It incorporates decision variables, an objective function, and several key constraints to ensure that all operational requirements are met.


In [4]:
def initialize_model():
    # Create the model
    model = gb.Model("MUHC Route Optimization")

    # We ask Gurobi not to print too much on screen
    model.Params.OutputFlag = 0
    
    return model

model = initialize_model()

Restricted license - for non-production use only - expires 2025-11-24


### Decision variables
- $ x_{ij}^k $: Binary variable indicating whether the truck travels from location $ i $ to location $ j $ in journey $ k $.
  $$
  x_{ij}^k = 
  \begin{cases} 
  1 & \text{if the truck travels from location } i \text{ to location } j \text{ in journey } k \\ 
  0 & \text{otherwise}
  \end{cases}
  $$

- $ u_j^k $: Integer variable representing the order in which the truck visits center $ j $ in journey $ k $.
  $$
  u_j^k \in \{1, 2, \dots, n-1\}, \quad \forall j \neq 0
  $$



In [5]:
def  add_decision_variables(model, K):
    # Define decision variables
    # Route indicator variable
    x = model.addVars(
        N,
        N,
        K,
        vtype=GRB.BINARY,
        name=(f"X[{i+1}, {j+1}, {k+1}]" for i in range(N) for j in range(N) for k in range(K)),
    )
    # Visiting order variable
    u = model.addVars(
        N,
        K,
        lb = 0,
        vtype=GRB.INTEGER,
        name=(f"U[{i+1}, {k+1}]" for i in range(N) for k in range(K)),
    )
    
    min_order = model.addVars(K, vtype=GRB.INTEGER, name="MinOrder")
    max_order = model.addVars(K, vtype=GRB.INTEGER, name="MaxOrder")
    
    return x, u, min_order, max_order, model


x, u, min_order, max_order, model = add_decision_variables(model, K)

### Objective function

The objective is to minimize the total cost, which includes both the distance and time traveled, and the priority of visiting each center:

$$
\min \left( \gamma \sum_{k=1}^{K} \sum_{i=1}^{N} \sum_{j=1}^{N} \left[\lambda_1 \frac{d_{ij}}{d_{\text{max}}} x_{ij}^k + \lambda_2 \frac{t_{ij}}{t_{\text{max}}} x_{ij}^k\right] + (1 - \gamma) \sum_{k=1}^{K} \sum_{j=1}^{N} p_j u_j^k \right)
$$

In [6]:
def set_objective(x, u, model, K, g, l):    
    # Objective function
    distance_opt = gb.quicksum(
        distance.iloc[i, j] * x[i, j, k] for i in range(N) for j in range(N) for k in range(K)
    )

    # within route prioritization
    priority_opt = gb.quicksum(
        priority[j] * u[j, k] for j in range(N) for k in range(K)
    )


    time_opt = gb.quicksum(
        travel_time_no_traffic.iloc[i, j] * x[i, j, k]  for i in range(N) for j in range(N) for k in range(K)
    )


    model.setObjective( g * l * distance_opt/max_dist + g * (1 - l) * time_opt/max_time + (1 - g) * priority_opt, sense=GRB.MINIMIZE)

    return model

model = set_objective(x, u, model, K, g, l)

### Constraints

#### 1. Demand Constraint:
For both volume and weight, we need to ensure that the total demand for each center $ j $ is satisfied across all journeys $ k $.

- **Volume**:
  $$
  \sum_{i=1}^{N} \sum_{k=1}^{K} x_{ij}^k \cdot \text{v}_j = \text{v}_j, \quad \forall j
  $$
  
- **Weight**:
  $$
  \sum_{i=1}^{N} \sum_{k=1}^{K} x_{ij}^k \cdot \text{w}_j = \text{w}_j, \quad \forall j
  $$

#### 2. Volume Capacity:
The total volume carried by the truck in each journey $ k $ must not exceed the truck's maximum volume capacity $ C_{\text{volume}} $.
  $$
  \sum_{i=1}^{N} \sum_{j=1}^{N} x_{ij}^k \cdot \text{v}_j \leq C_{\text{volume}}, \quad \forall k
  $$

#### 3. Weight Capacity:
The total weight carried by the truck in each journey $ k $ must not exceed the truck's maximum weight capacity $ C_{\text{weight}} $.
  $$
  \sum_{i=1}^{N} \sum_{j=1}^{N} x_{ij}^k \cdot \text{w}_j \leq C_{\text{weight}}, \quad \forall k
  $$

#### 4. Each Journey Must Start at Glen:
Each journey $ k $ must start at the depot (Glen, denoted by index 0).
  $$
  \sum_{j=1}^{N} x_{0j}^k = 1, \quad \forall k
  $$

#### 5. Each Journey Must End at Glen:
Each journey $ k $ must end at the depot (Glen).
  $$
  \sum_{i=1}^{N} x_{i0}^k = 1, \quad \forall k
  $$

#### 6. Must Visit Each Location Exactly Once:
Each center $ j $ must be visited exactly once across all journeys $ k $, excluding the depot.
  $$
  \sum_{i=1}^{N} \sum_{k=1}^{K} x_{ij}^k = 1, \quad \forall j \neq \text{0}
  $$

#### 7. Must Exit Each Location Exactly Once:
From each center $ i $, the truck must exit exactly once across all journeys $ k $, excluding the depot.
  $$
  \sum_{j=1}^{N} \sum_{k=1}^{K} x_{ij}^k = 1, \quad \forall i \neq \text{0}
  $$

#### 8. Subtour Elimination:
To eliminate subtours, we need to enforce the following constraint for all journeys $ k $, centers $ i $ and $ j $ (excluding the depot):
  $$
  u_i^k - u_j^k + N \cdot x_{ij}^k \leq N - 1, \quad \forall i \neq j, i \neq \text{0}, j \neq \text{0}
  $$

#### 9. Max/Min Order for Nodes in Journey K:
To ensure that the order for a node in journey K is not greater than the maximum order or less than the minimum order
$$
MaxOrder_{k} \ge u_{ik}, \quad \forall i, \forall k
$$

$$
MinOrder_{k} \le u_{ik}, \quad \forall i, \forall k
$$

#### 10. Min order for journey K+1 must be greater than or equal to max order for journey K
$$
MinOrder_{k+1} \ge MaxOrder_{k}, \quad \forall k-1
$$

#### 11. Flow Conservation:
The number of trucks arriving at a center must equal the number of trucks leaving that center for all centers $ i $ (excluding the depot) and all journeys $ k $:
  $$
  \sum_{j=1}^{N} x_{ij}^k = \sum_{j=1}^{N} x_{ji}^k, \quad \forall i \neq \text{0}, \forall k
  $$

#### 12. Self-routing Elimination:
The truck cannot travel from a center $ i $ to itself in any journey $ k $, excluding the depot.
  $$
  x_{ii}^k = 0, \quad \forall i \neq \text{0}, \forall k
  $$



In [7]:
    
def add_constraints(x, u, model, K, max_volume, max_load):
    # Constraints 1: Demand Constraint
    model.addConstrs(
        (gb.quicksum(x[i, j, k] * volume_cuft[j] for i in range(N) for k in range(K)) == volume_cuft[j]
                    for j in range(N)),
        name="Demand Constraint (vol)",
    )

    model.addConstrs(
        (gb.quicksum(x[i, j, k] * load_lbs[j] for i in range(N) for k in range(K)) == load_lbs[j]
                    for j in range(N)),
        name="Demand Constraint (weight)",
    )

    # Constraint 2: Volume Capacity
    model.addConstrs(
        (gb.quicksum(x[i, j, k] * volume_cuft[j] for i in range(N) for j in range(N)) <= max_volume
                    for k in range(K)),
        name="Volume Capacity",
    )

    # Constraint 3: Weight Capacity
    model.addConstrs(
        (gb.quicksum(x[i, j, k] * load_lbs[j] for i in range(N) for j in range(N)) <= max_load 
                    for k in range(K)),
        name="Weight Capacity",
    )

    # Constraint 4: Each journey must start at Glen
    model.addConstrs(
        (gb.quicksum(x[START_NODE_IDX, j, k] for j in range(N)) == 1 
                    for k in range(K)),
        name="Each journey must end at Glen",
    )

    # Constraint 5: Each journey must end at Glen
    model.addConstrs(
        (gb.quicksum(x[i, START_NODE_IDX, k] for i in range(N)) == 1 
                    for k in range(K)),
        name="Each journey must end at Glen",
    )

    # Constraint 6: Must visit every location exactly once
    model.addConstrs(
        (gb.quicksum(x[i, j, k] for i in range(N) for k in range(K)) == 1 
                    for j in range(N) 
                    if j != START_NODE_IDX),
        name="Must visit every location exactly once",
    )

    # Constraint 7: Must exit every location exactly once
    model.addConstrs(
        (gb.quicksum(x[i, j, k] for j in range(N) for k in range(K)) == 1 
                    for i in range(N) 
                    if i != START_NODE_IDX),
        name="Must exit every location exactly once",
    )

    # Constraint 8: No subtours
    model.addConstrs(
        (
            u[i,k] - u[j,k] + N * x[i, j, k] <= N - 1
            for i in range(1,N)
            for j in range(1,N)
            for k in range(K)
            if i != j and i != START_NODE_IDX and j != START_NODE_IDX
        ),
        name="Subtour elimination constraint",
    )
    
    #constraint #9: Max/min order constraint for journey k
    for k in range(K):
        for i in range(1, N):
            # Maximum order constraint for journey k
            model.addConstr(
                max_order[k] >= u[i, k],  # Max order is at least as large as the order of any node in the journey
                name=f"MaxOrder_{i}_{k}"
            )
            
            # Minimum order constraint for journey k
            model.addConstr(
                min_order[k] <= u[i, k],  # Min order is at most as small as the order of any node in the journey
                name=f"MinOrder_{i}_{k}"
            )
    
    #constraint #10: Journey K+1 orders must be greater than Journey K Order Constraint
    for k in range(K-1):
        model.addConstr(
            min_order[k + 1] >= max_order[k],  # Min order in journey k+1 must be greater than max order in journey k
            name=f"OrderAcrossJourneys_{k}"
        )

    # Constraint 11: Flow conservation
    model.addConstrs(
        (gb.quicksum(x[i, j, k] for j in range(N) if j != i) == gb.quicksum(x[j, i, k] for j in range(N) if j != i)
        for i in range(START_NODE_IDX+1, N)  # Node i != 0 (i.e., not the depot)
        for k in range(K)),    # For each journey
        name="Flow_Conservation"
    )
    # Constraint 12 : Self routing elimination
    model.addConstrs(
        (x[i, i, k] == 0 for i in range(N) for k in range(K) if i != START_NODE_IDX), 
        name="Self-routing elimination"
    )

    return model

model = add_constraints(x, u, model, K, max_volume, max_load)

   

### Solving the Baseline Model and Results
Once the model is fully defined, the Gurobi optimizer is employed to solve the optimization problem. Gurobi efficiently computes the optimal solution by analyzing the defined decision variables, objective function, and constraints, ensuring that all requirements are met while optimizing the key objectives.

The process begins with the definition of decision variables—such as which routes to take and in what order. Next, the objective function is set, prioritizing the minimization of both distance and travel time, while also accounting for the priority of different healthcare sites. The optimizer then evaluates various routing possibilities, taking into account all imposed constraints, such as truck capacity, site demands, and the necessity to visit each location only once.

In [8]:
def optimize_model(model):
    # Print out the whole model
    model.update()
    model.display()
    model.optimize()

    return model

model = optimize_model(model)

C:\Users\Collard\AppData\Local\Temp\ipykernel_20952\1690223594.py:4: DeprecationWarning: Model.display() is deprecated
  model.display()


In [9]:
route_dic = {}

def obtain_results(model, K):

    global route_dic  # Use the global route_dic inside the function
# Check if the model found an optimal solution
    if model.Status == GRB.OPTIMAL:
        
        # Print the optimal cost
        print(f"The optimal cost is: {round(model.objVal, 2)}")

        # Print the optimal route for each journey
        print("\nThe optimal route is:")

        route_num = 1

        for k in range(K):  # Iterate over each journey
            current_location = START_NODE_IDX  # Start from the depot (index 0)
            route = [current_location]  # Start the route with the depot
            
            visited = set()  # Track visited centers to prevent loops
            visited.add(current_location)

            # Follow the path taken by the truck
            for _ in range(N):  # Limit the loop to avoid infinite loops (based on the number of centers)
                found_next = False
                for j in range(N):
                    if x[current_location, j, k].x > 0.5 and j not in visited:  # If truck moves to an unvisited center
                        route.append(j)  # Append the next center to the route
                        visited.add(j)  # Mark it as visited
                        current_location = j  # Move to the next center
                        found_next = True
                        break

                if not found_next:  # If no more destinations, break (the truck has returned to the depot)
                    current_route = []
                    route_num = route_num
                    for j in route:
                        current_route.append(j)
                    if current_route[-1] == 0:
                        continue
                    else: 
                        current_route.append(0)
                        route_dic[f"{route_num}"] = current_route
                        route_num += 1
                    break

            # Display the route in a single line, starting from the depot
            if len(route) > 1:
                print(f"Journey {k+1}: Depot -> ", end="")
                for i, node in enumerate(route[1:]):  # Skip the starting depot in the display
                    print(f"{node}", end="")
                    if i < len(route[1:]) - 1:
                        print(" -> ", end="")
                print(f" -> Depot")  # Return to the depot
                

        # The following code is to confirm that all the merchandise was distributed as expected
        # Commented out for final output, but used for testing and development 
        """
        # Calculate the total volume and weight assigned
        total_assigned_volume = 0
        total_assigned_weight = 0

        for i in range(N):
            for j in range(N):
                for k in range(K):
                    if x[i, j, k].x > 0.5:  # If the route from i to j in journey k was taken
                        total_assigned_volume += volume_cuft[j]
                        total_assigned_weight += load_lbs[j]

        # Calculate the leftover volume and weight not distributed
        leftover_volume = sum(volume_cuft) - total_assigned_volume
        leftover_weight = sum(load_lbs) - total_assigned_weight

        # Display the results for volume and weight
        print(f"\nTotal volume assigned: {total_assigned_volume} cu ft")
        print(f"Total weight assigned: {total_assigned_weight} lbs")
        print(f"\nLeftover volume not distributed: {leftover_volume} cu ft")
        print(f"Leftover weight not distributed: {leftover_weight} lbs")
        """
        
    else:
        print("No optimal solution found.")

    return model

print ("\n************************************")
print(f"Results for baseline model:")
model = obtain_results(model, K)


************************************
Results for baseline model:
The optimal cost is: 5.36

The optimal route is:
Journey 1: Depot -> 1 -> Depot
Journey 2: Depot -> 5 -> 4 -> 3 -> 2 -> Depot


### Visual Depiction of Optimal Route for Baseline Model
We utilize the Folium library to create an interactive map that visualizes the optimized delivery routes for the MUHC logistics network. Each healthcare site is marked by its specific location, with the optimized routes connecting these sites clearly displayed. This visual representation makes it easy to interpret the routing solution and understand the efficiency gains and logistics improvements.

Markers are used to pinpoint each healthcare site, while lines are drawn to represent the optimized routes between them. This visualization allows stakeholders to quickly grasp how the optimized routes help in reducing travel distance, minimizing time, and lowering operational costs.

The interactive nature of the map adds another layer of usability by allowing users to explore the optimized routes in more detail. Stakeholders can zoom in, review the paths between specific sites, and even assess areas where bottlenecks might occur. This makes it a user-friendly tool for evaluating the efficiency of the MUHC logistics network and for identifying areas where further improvements could be made.



In [10]:
# Create a separate instance of x
x_original = x.copy()  # Assuming x is a gurobipy.tupledict

m2 = fl.Map(location=coordinates[0], zoom_start=12)

for name, loc in coordinates.items():
    fl.Marker(location=loc, popup=name).add_to(m2)

for loc in coordinates:
    text = loc_name[loc]
    lat = coordinates[loc][0]
    lon = coordinates[loc][1]
    
    fl.map.Marker(
        [lat + 0.001, lon + 0.001],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 12pt; font-weight: bold;"><b>%s</b></div>' % text,  # <b> added for bold
            )
        ).add_to(m2)

# Get the 'cool' colormap
cmap = plt.get_cmap('cool')

# Number of colors to pick based on the length of route_dic
n = len(route_dic)

# Generate `n` distinct colors from the colormap
icon_colors = [mcolors.to_hex(cmap(i / (n-1))) for i in range(n)]

# Iterate over route_dic
for x, (key, route) in enumerate(route_dic.items()):
    nth_locations = []
    
    # Append the coordinates for the route
    for i in route:
        nth_locations.append(coordinates[i])
    
    # Get the color corresponding to the current route
    icon_color = icon_colors[x]
    
    # Prepare OSRM coordinates
    osrm_coords = ";".join([f"{lon},{lat}" for lat, lon in nth_locations])
    
    # OSRM server endpoint
    osrm_url = f"http://router.project-osrm.org/route/v1/driving/{osrm_coords}?overview=full&geometries=geojson"
    
    # Make the request to OSRM API
    response = requests.get(osrm_url)
    data = response.json()
    
    # Extract the route geometry (a list of lat-long points)
    route_geometry = data['routes'][0]['geometry']['coordinates']
    route_points = [(lat, lon) for lon, lat in route_geometry]  # Reverse to folium format (lat, lon)
    
    fl.PolyLine(locations=route_points, color=icon_color,weight=5).add_to(m2)
    
m2

## Post Optimization Breakdown of Baseline Model

In this analysis, we aim to evaluate the efficiency improvements achieved after implementing the optimized vehicle routing model for the MUHC logistics network. Using baseline values representative of the current logistics process at MUHC, we compare key metrics such as total distance traveled, time spent, labor costs, and CO2 emissions against those resulting from the optimized model. This analysis highlights the savings in distance, time, costs, and environmental impact, illustrating the value added by optimizing the vehicle routing operations.

The following metrics are included in the post-optimization breakdown:
- **Distance**: Comparison of total kilometers traveled before and after optimization, along with the associated cost savings.
- **Time**: Comparison of total time spent before and after optimization, as well as labor cost savings.
- **CO2 Emissions**: Estimation of CO2 emissions savings based on total distance reductions.


In [11]:
x = x_original

def distance_component_savings(x, K, distance, baseline_distance, cost_per_km=1.25): #cost_per_km takes an average of the depreciation, insurrance, fuel cost
    total_distance_km = 0
    total_cost_distance = 0

    for i in range(distance.shape[0]):
        for j in range(distance.shape[0]):
            for k in range(K):
                if x[i, j, k].x > 0.5:  # Route from i to j taken in journey k
                    total_distance_km += distance.iloc[i, j]
                    total_cost_distance += distance.iloc[i, j] * cost_per_km

    # Calculate savings compared to the baseline
    distance_saved_km = baseline_distance - total_distance_km
    cost_saved_distance = distance_saved_km * cost_per_km

    return total_distance_km, distance_saved_km, cost_saved_distance

def time_component_savings(x, K, travel_time_no_traffic, baseline_time_min, wage_per_minute=0.43):
    total_time_min = 0

    for i in range(travel_time_no_traffic.shape[0]):
        for j in range(travel_time_no_traffic.shape[0]):
            for k in range(K):
                if x[i, j, k].x > 0.5:
                    total_time_min += travel_time_no_traffic.iloc[i, j]

    # Calculate savings compared to the baseline
    time_saved_min = baseline_time_min - total_time_min
    cost_saved_time = time_saved_min * wage_per_minute

    return total_time_min, time_saved_min, cost_saved_time

def labor_component(total_time_min, wage_per_minute=0.43):
    total_cost_time = total_time_min * wage_per_minute
    return total_cost_time

def co2_emissions_component_savings(x, K, distance, baseline_co2_emission, co2_per_km=0.3):
    total_co2_emission = 0

    for i in range(distance.shape[0]):
        for j in range(distance.shape[0]):
            for k in range(K):
                if x[i, j, k].x > 0.5:
                    total_co2_emission += distance.iloc[i, j] * co2_per_km

    # Calculate savings compared to the baseline
    co2_saved = baseline_co2_emission - total_co2_emission

    return total_co2_emission, co2_saved

# Baseline values for each component, which is reliscaly the process at MUHC, based on manager expertise at the logistics center
baseline_distance = 150
baseline_time_min = 420  # a 7 hours shift 
baseline_co2_emission = 45 #0.3*distance 


total_distance_km, distance_saved_km, cost_saved_distance = distance_component_savings(x, K, distance, baseline_distance)
total_time_min, time_saved_min, cost_saved_time = time_component_savings(x, K, travel_time_no_traffic, baseline_time_min)
total_co2_emission, co2_saved = co2_emissions_component_savings(x, K, distance, baseline_co2_emission)


def display_post_optimization_results_with_baseline(baseline_distance, total_distance_km, distance_saved_km, cost_saved_distance,
                                                    baseline_time_min, total_time_min, time_saved_min, cost_saved_time,
                                                    baseline_co2_emission, total_co2_emission, co2_saved):
    print(f"\n--- Post Optimization Savings with Baseline ---\n")

    print(f"Distance Component:")
    print(f"    Total Distance without Optimization (Baseline at MUHC) (km): {baseline_distance:.1f}")
    print(f"    Total Distance with Optimization (km): {total_distance_km:.1f}")
    print(f"    Distance Saved with Optimization (km): {distance_saved_km:.1f}")
    print(f"    Cost Saved Related to Distance ($): {cost_saved_distance:.1f}")

    print(f"\nTime Component:")
    print(f"    Total Time without Optimization (Baseline at MUHC) (min): {baseline_time_min:.1f}")
    print(f"    Total Time with Optimization (min): {total_time_min:.1f}")
    print(f"    Time Saved with Optimization (min): {time_saved_min:.1f}")
    print(f"    Labor Cost Saved ($): {cost_saved_time:.1f}")

    print(f"\nCO2 Emissions Component:")
    print(f"    Total CO2 Emissions without Optimization (Baseline at MUHC) (kg): {baseline_co2_emission:.1f}")
    print(f"    Total CO2 Emissions with Optimization (kg): {total_co2_emission:.1f}")
    print(f"    CO2 Emissions Saved with Optimization (kg): {co2_saved:.1f}")


display_post_optimization_results_with_baseline(
    baseline_distance, total_distance_km, distance_saved_km, cost_saved_distance,
    baseline_time_min, total_time_min, time_saved_min, cost_saved_time,
    baseline_co2_emission, total_co2_emission, co2_saved
)


--- Post Optimization Savings with Baseline ---

Distance Component:
    Total Distance without Optimization (Baseline at MUHC) (km): 150.0
    Total Distance with Optimization (km): 63.5
    Distance Saved with Optimization (km): 86.5
    Cost Saved Related to Distance ($): 108.1

Time Component:
    Total Time without Optimization (Baseline at MUHC) (min): 420.0
    Total Time with Optimization (min): 80.4
    Time Saved with Optimization (min): 339.6
    Labor Cost Saved ($): 146.0

CO2 Emissions Component:
    Total CO2 Emissions without Optimization (Baseline at MUHC) (kg): 45.0
    Total CO2 Emissions with Optimization (kg): 19.1
    CO2 Emissions Saved with Optimization (kg): 25.9


### Sensitivity Analysis

In addition to solving the baseline model, we conduct a **comprehensive sensitivity analysis** to see how changing key parameters affects the model’s performance and robustness. Here's how we approach it:

**Gamma Sensitivity**  
We adjust the parameter **γ** to explore the balance between minimizing total operational costs (distance and time) and prioritizing more critical healthcare sites.

- A **lower γ value** emphasizes the priority of servicing specific sites, ensuring higher-priority locations are visited first.
- A **higher γ value** focuses more on cost efficiency, reducing overall distance and time.

This allows us to understand the trade-off between **cost minimization** and **service level adherence** based on site priority.

**Lambda Sensitivity**  
The parameter **λ** helps us analyze the impact of focusing on **time** vs. **distance** in the optimization.

- **λ₁** represents the weight assigned to minimizing distance.
- **λ₂** focuses on minimizing travel time.

By testing different combinations of **λ values**, we can assess the trade-offs between faster deliveries and shorter routes. This is key for aligning with specific goals, such as **fuel savings** or **reducing delivery delays**.

**Truck Size Sensitivity**  
We evaluate how using **different truck sizes** affects costs and route efficiency.

- By changing truck capacities (both **volume and weight**), we assess how larger or smaller trucks influence the **number of trips**, **distance traveled**, and **total costs**.
- This analysis helps us understand how fleet composition impacts **operational costs** and **efficiency**, which is especially useful when scaling the fleet to match demand.


This analysis gives us a deeper understanding of the model’s robustness and helps guide operational strategies. By adjusting factors like **fleet composition**, **site prioritization**, and **time vs. distance optimization**, we gain insights into how these strategies can improve the **efficiency** and **cost-effectiveness** of the MUHC logistics network.


In [12]:
# Impact of assigning different weights for cost vs priority, keeping everything else constant
model_gamma = []

# constants
l = 0.75
max_volume = truck_data["max_volume"][0]
max_load = truck_data["max_load"][0]

# gamma
gamma = [0, 0.25, 0.75, 1]

for gs in gamma:
    # Going through every step of optimization process
    K = obtain_k(max_volume,max_load)
    model = initialize_model()
    x, u, min_order, max_order, model = add_decision_variables(model, K)
    model = set_objective(x, u, model, K, gs, l)
    model = add_constraints(x, u, model, K, max_volume, max_load)
    model_gamma.append(optimize_model(model))

    print ("\n************************************")
    print(f"Results when gamma is set to be: {gs}")
    model = obtain_results(model, K)


************************************
Results when gamma is set to be: 0
The optimal cost is: 6.0

The optimal route is:
Journey 1: Depot -> 1 -> Depot
Journey 2: Depot -> 5 -> 2 -> 4 -> 3 -> Depot

************************************
Results when gamma is set to be: 0.25
The optimal cost is: 5.36

The optimal route is:
Journey 1: Depot -> 1 -> Depot
Journey 2: Depot -> 5 -> 4 -> 3 -> 2 -> Depot

************************************
Results when gamma is set to be: 0.75
The optimal cost is: 4.08

The optimal route is:
Journey 1: Depot -> 1 -> Depot
Journey 2: Depot -> 5 -> 4 -> 3 -> 2 -> Depot


C:\Users\Collard\AppData\Local\Temp\ipykernel_20952\1690223594.py:4: DeprecationWarning: Model.display() is deprecated
  model.display()



************************************
Results when gamma is set to be: 1
The optimal cost is: 3.16

The optimal route is:
Journey 1: Depot -> 2 -> 1 -> 4 -> Depot
Journey 2: Depot -> 5 -> 3 -> Depot


In [13]:
# Impact of assigning different weights for distance vs time, keeping everything else constant
model_lambda = []

# constants
g = 0.25
max_volume = truck_data["max_volume"][0]
max_load = truck_data["max_load"][0]

lambda_ = [0, 0.25, 0.75, 1]

for ls in lambda_:

    # Going through every step of optimization process
    K = obtain_k(max_volume,max_load)
    model = initialize_model()
    x, u, min_order, max_order, model = add_decision_variables(model, K)
    model = set_objective(x, u, model, K, g, ls)
    model = add_constraints(x, u, model, K, max_volume, max_load)
    model_lambda.append(optimize_model(model))

    print ("\n************************************")
    print(f"Results when lambda is set to be: {ls}")
    model = obtain_results(model, K)


************************************
Results when lambda is set to be: 0
The optimal cost is: 5.43

The optimal route is:
Journey 1: Depot -> 1 -> Depot
Journey 2: Depot -> 5 -> 3 -> 4 -> 2 -> Depot

************************************
Results when lambda is set to be: 0.25
The optimal cost is: 5.41

The optimal route is:
Journey 1: Depot -> 1 -> Depot
Journey 2: Depot -> 5 -> 4 -> 3 -> 2 -> Depot

************************************
Results when lambda is set to be: 0.75
The optimal cost is: 5.36

The optimal route is:
Journey 1: Depot -> 1 -> Depot
Journey 2: Depot -> 5 -> 4 -> 3 -> 2 -> Depot

************************************
Results when lambda is set to be: 1
The optimal cost is: 5.34

The optimal route is:
Journey 1: Depot -> 1 -> Depot
Journey 2: Depot -> 5 -> 4 -> 3 -> 2 -> Depot


C:\Users\Collard\AppData\Local\Temp\ipykernel_20952\1690223594.py:4: DeprecationWarning: Model.display() is deprecated
  model.display()


In [14]:
# Impact of using different size truck, keeping everything else constant
model_truck = []

# constants
l = 0.75
g = 0.75

for i in range(len(truck_data["truck"])):

    max_volume_s = truck_data["max_volume"][i]
    max_load_s = truck_data["max_load"][i]

    # Going through every step of optimization process
    K = obtain_k(max_volume_s,max_load_s)
    model = initialize_model()
    x, u, min_order, max_order, model = add_decision_variables(model, K)
    model = set_objective(x, u, model, K, g, l)
    model = add_constraints(x, u, model, K, max_volume_s, max_load_s)
    model_truck.append(optimize_model(model))

    print ("\n************************************")
    print(f"Results for truck size {truck_data['truck'][i]}:")
    model = obtain_results(model, K)

C:\Users\Collard\AppData\Local\Temp\ipykernel_20952\1690223594.py:4: DeprecationWarning: Model.display() is deprecated
  model.display()



************************************
Results for truck size 10':
The optimal cost is: 4.08

The optimal route is:
Journey 1: Depot -> 1 -> Depot
Journey 2: Depot -> 5 -> 4 -> 3 -> 2 -> Depot

************************************
Results for truck size 15':
The optimal cost is: 4.7

The optimal route is:
Journey 1: Depot -> 5 -> 1 -> 4 -> 3 -> 2 -> Depot

************************************
Results for truck size 17':
The optimal cost is: 4.7

The optimal route is:
Journey 1: Depot -> 5 -> 1 -> 4 -> 3 -> 2 -> Depot

************************************
Results for truck size 20':
The optimal cost is: 4.7

The optimal route is:
Journey 1: Depot -> 5 -> 1 -> 4 -> 3 -> 2 -> Depot

************************************
Results for truck size 26':
The optimal cost is: 4.7

The optimal route is:
Journey 1: Depot -> 5 -> 1 -> 4 -> 3 -> 2 -> Depot


### Solving the Adjusted Model and Results
Using carefully selected parameters from the sensitivity analysis, including weights for lambda, gamma, and truck capacity, we then solve the model based on these ideal values to determine a likely best-case scenario. This approach allows us to fine-tune the optimization by adjusting these parameters to reflect their relative importance in the model. Lambda and gamma control trade-offs between competing objectives for distance and time as well as cost and priority, while truck volume and weight constraints ensure feasible and realistic outcomes. By solving the model with these ideal parameters, we obtain insights into how the system performs under optimal conditions, which can guide decision-making in practice.

In [15]:
# ideal parameters
max_volume = truck_data["max_volume"][1] # favours 17' truck
max_load = truck_data["max_load"][1] # favours 17' truck
l = 1 # favours economical distance over time
g = 1 # favours cost over priority


print ("\n************************************")
print(f"Results for adjusted model:")

model = initialize_model()
x, u, min_order, max_order, model = add_decision_variables(model, K)
model = set_objective(x, u, model, K, g, l)
model = add_constraints(x, u, model, K, max_volume, max_load)
model = optimize_model(model)
model = obtain_results(model, K)



************************************
Results for adjusted model:
The optimal cost is: 2.37

The optimal route is:
Journey 1: Depot -> 2 -> 3 -> 4 -> 1 -> 5 -> Depot


C:\Users\Collard\AppData\Local\Temp\ipykernel_20952\1690223594.py:4: DeprecationWarning: Model.display() is deprecated
  model.display()


## Post Optimization Breakdown of Adjusted Model
Just as with the analysis of the baseline model, we conduct the same post-optimization breakdown of our adjusted model below.

The following metrics are included in the post-optimization breakdown:
- **Distance**: Comparison of total kilometers traveled before and after optimization, along with the associated cost savings.
- **Time**: Comparison of total time spent before and after optimization, as well as labor cost savings.
- **CO2 Emissions**: Estimation of CO2 emissions savings based on total distance reductions.


In [16]:
# Baseline values for each component, which is realistically the process at MUHC, based on manager expertise at the logistics center
baseline_distance = 150
baseline_time_min = 420  # a 7 hours shift 
baseline_co2_emission = 45 # 0.3 * distance 


total_distance_km, distance_saved_km, cost_saved_distance = distance_component_savings(x, K, distance, baseline_distance)
total_time_min, time_saved_min, cost_saved_time = time_component_savings(x, K, travel_time_no_traffic, baseline_time_min)
total_co2_emission, co2_saved = co2_emissions_component_savings(x, K, distance, baseline_co2_emission)


def display_post_optimization_results_with_baseline(baseline_distance, total_distance_km, distance_saved_km, cost_saved_distance,
                                                    baseline_time_min, total_time_min, time_saved_min, cost_saved_time,
                                                    baseline_co2_emission, total_co2_emission, co2_saved):
    print(f"\n--- Post Optimization Savings with Adjusted ---\n")

    print(f"Distance Component:")
    print(f"    Total Distance without Optimization (Baseline at MUHC) (km): {baseline_distance}")
    print(f"    Total Distance with Optimization (km): {total_distance_km}")
    print(f"    Distance Saved with Optimization (km): {distance_saved_km}")
    print(f"    Cost Saved Related to Distance ($): {cost_saved_distance}")

    print(f"\nTime Component:")
    print(f"    Total Time without Optimization (Baseline at MUHC) (min): {baseline_time_min}")
    print(f"    Total Time with Optimization (min): {total_time_min}")
    print(f"    Time Saved with Optimization (min): {time_saved_min}")
    print(f"    Labor Cost Saved ($): {cost_saved_time}")

    print(f"\nCO2 Emissions Component:")
    print(f"    Total CO2 Emissions without Optimization (Baseline at MUHC) (kg): {baseline_co2_emission}")
    print(f"    Total CO2 Emissions with Optimization (kg): {total_co2_emission}")
    print(f"    CO2 Emissions Saved with Optimization (kg): {co2_saved}")


display_post_optimization_results_with_baseline(
    baseline_distance, total_distance_km, distance_saved_km, cost_saved_distance,
    baseline_time_min, total_time_min, time_saved_min, cost_saved_time,
    baseline_co2_emission, total_co2_emission, co2_saved
)




--- Post Optimization Savings with Adjusted ---

Distance Component:
    Total Distance without Optimization (Baseline at MUHC) (km): 150
    Total Distance with Optimization (km): 45.0
    Distance Saved with Optimization (km): 105.0
    Cost Saved Related to Distance ($): 131.25

Time Component:
    Total Time without Optimization (Baseline at MUHC) (min): 420
    Total Time with Optimization (min): 58.8
    Time Saved with Optimization (min): 361.2
    Labor Cost Saved ($): 155.316

CO2 Emissions Component:
    Total CO2 Emissions without Optimization (Baseline at MUHC) (kg): 45
    Total CO2 Emissions with Optimization (kg): 13.5
    CO2 Emissions Saved with Optimization (kg): 31.5
